# A. Enfoque de validación
##ACA ESTAN LAS BASES DE DATOS Y UNA MUESTRA DE QUE ESTAN BIEN

In [20]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [21]:
norespondieron = pd.read_excel("norespondieron.xlsx")
respondieron = pd.read_excel("respondieron.xlsx")
print(norespondieron.shape)
print(respondieron.shape)
print(norespondieron.columns.tolist())
print(respondieron.columns.tolist())

(2985, 23)
(12866, 26)
['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'REGION', 'ANO4', 'CH04', 'CH06', 'CH07', 'CH08', 'CH12', 'CH13', 'CH14', 'NIVEL_ED', 'ESTADO', 'CAT_INAC', 'PP10A', 'PP03J', 'PP04A', 'P21', 'P47T', 'ITF', 'IPCF', 'adulto_equiv']
['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'REGION', 'ANO4', 'CH04', 'CH06', 'CH07', 'CH08', 'CH12', 'CH13', 'CH14', 'NIVEL_ED', 'ESTADO', 'CAT_INAC', 'PP10A', 'PP03J', 'PP04A', 'P21', 'P47T', 'ITF', 'IPCF', 'adulto_equiv', 'ad_equiv_hogar', 'ingreso_necesario', 'pobre']


In [ ]:
# Obtener los tipos de datos de cada df
tipos_no = norespondieron.dtypes.rename("norespondieron")
tipos_re = respondieron.dtypes.rename("respondieron")

# Unirlos en un solo DataFrame
comparacion_tipos = pd.concat([tipos_no, tipos_re], axis=1)

# Mostrar el cuadro resultante
print(comparacion_tipos)

                  norespondieron respondieron
CODUSU                    object       object
NRO_HOGAR                  int64        int64
COMPONENTE                 int64        int64
REGION                     int64        int64
ANO4                       int64        int64
CH04                       int64        int64
CH06                       int64        int64
CH07                       int64        int64
CH08                       int64        int64
CH12                       int64        int64
CH13                       int64        int64
CH14                       int64        int64
NIVEL_ED                   int64        int64
ESTADO                     int64        int64
CAT_INAC                   int64        int64
PP10A                      int64        int64
PP03J                      int64        int64
PP04A                      int64        int64
P21                        int64        int64
P47T                       int64        int64
ITF                        int64  

In [32]:
# creamos las variables a usar, la variables dependiente y variables independientes
variables_X = ['ANO4', 'CH04','CH06','CH07', 'REGION','ESTADO']
variable_y = 'pobre'

# Crear X e y
X = respondieron[variables_X].copy()
y = respondieron[variable_y]

# Agregar columna de unos para el intercepto
X.insert(0, 'const', 1)

In [33]:
# Separar en train y test (70%-30%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=444
)

###ejercicio 1

In [34]:
# Calcular medias en train y test
media_train = X_train.mean()
media_test = X_test.mean()

In [35]:
# Calcular diferencia de medias
diferencias = media_train - media_test

In [36]:
# Crear tabla resumen
tabla_dif = pd.DataFrame({
    'Media_Train': media_train,
    'Media_Test': media_test,
    'Diferencia': diferencias
}).round(3)

print(tabla_dif)

        Media_Train  Media_Test  Diferencia
const         1.000       1.000       0.000
ANO4       2011.587    2011.352       0.234
CH04          1.527       1.520       0.007
CH06         35.880      36.033      -0.153
CH07          3.463       3.474      -0.011
REGION        1.000       1.000       0.000
ESTADO        2.201       2.201      -0.000
